In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Implimenting K-mean 

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [ ]:
X[:5,:]
#print(X.shape)

In [ ]:
y[:5]
print(y.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.scatter(X[:,0], X[:,1], c = y)
plt.show()

In [ ]:
K = 3
random_indices = np.random.choice(X.shape[0], K)
centroids = X[random_indices]
centroids

In [ ]:
def visualize_centroids(X, centroids):
    plt.scatter(X[:,2], X[:,3])
    plt.scatter(centroids[:,0], centroids[:,1], marker = '*', s=200, c='red')
    plt.show()
visualize_centroids(X, centroids)    

In [ ]:
def dist(a,b):
    return np.linalg.norm(a-b, axis = 1)

def assign_cluster(x, centroids):
    distances = dist(x, centroids)
    cluster = np.argmin(distances)
    return cluster

def update_centroids(X, centroids, clusters):
    for i in range(len(centroids)):
        cluster_i = np.where(clusters == i)
        centroids[i] = np.mean(X[cluster_i], axis = 0)

In [ ]:
tor = 0.00001
K = 3
max_iter = 100
clusters = np.zeros(X.shape[0])
random_indices = np.random.choice(len(X), K)
centroids = X[random_indices]

In [ ]:
for itr in range(max_iter):
    prev_centroids = centroids.copy()
    # update clusters
    for i in range(len(X)):
        clusters[i] = assign_cluster(X[i], centroids)
    # update centroid    
    update_centroids(X, centroids, clusters)
    # check improvement
    centroids_diff = np.linalg.norm(centroids - prev_centroids)
    if centroids_diff < tor:
        print('\n------------')
        print("The algorithm converges after {0} iterations with centroids_diff = {1}".format(itr, centroids_diff))
        print('The centroids are: \n')
        print(centroids)
        break
    else:
        print('\n------------')
        print('Iteration {0}: centroids_diff = {1} \n'.format(itr, centroids_diff))
        print(centroids)
        
        


In [ ]:
visualize_centroids(X, centroids)

In [ ]:
from sklearn.cluster import KMeans
kmean_sk = KMeans(n_clusters = 3, random_state = 42)

In [ ]:
kmean_sk.fit(X)

In [ ]:
clusters_sk = kmean_sk.labels_
clusters_sk

In [ ]:
labels = np.where(clusters_sk == 1)[0]
labels

In [ ]:
np.sum(np.linalg.norm(X[labels] - centroids[0], axis = 1))

In [ ]:
centroids_sk = kmean_sk.cluster_centers_
centroids_sk

In [ ]:
visualize_centroids(X, centroids_sk)

## Implement KMean class

In [ ]:
class MyKmean():
    def __init__(self, k, n = 100, tor = 0.0001):
        self.n_clusters = k
        self.n_iters = n
        self.tor = tor
        
    def fit(self, X):
        random_indices = np.random.choice(len(X), self.n_clusters)
        self.centroids = X[random_indices]
        self.clusters = np.zeros(len(X))
        
        for itr in range(self.n_iters):
            for i in range(len(X)):
                self.clusters[i] = self.assign_cluster(X[i], self.centroids)
            self.update_centroids(X,self.centroids, self.clusters)
            # check termination 
            centroids_diff = np.linalg.norm(self.centroids - prev_centroids)
            if centroids_diff < tor:
                return self
        return self
            
                
            
        
    def dist(self, a,b):
        return np.linalg.norm(a-b, axis = 1)
    
    def assign_cluster(self, x, centroids):
        distances = self.dist(x, centroids)
        cluster = np.argmin(distances)
        return cluster

    def update_centroids(self, X, centroids, clusters):
        for i in range(len(centroids)):
            cluster_i = np.where(clusters == i)
            centroids[i] = np.mean(X[cluster_i], axis = 0)

In [ ]:
clf = MyKmean(k = 3)
clf.fit(X)

In [ ]:
clf.centroids

## Elbow method

In [ ]:
k_list = list(range(1,7))
sse_list = []

In [ ]:
def sse(X, y, centroids):
    sse = 0
    for i in range(len(centroids)):
        cluster_i = np.where(y==i)[0]
        sse += np.sum(np.linalg.norm(X[cluster_i] - centroids[i], axis = 1))
        
    return sse

In [ ]:
for k in k_list:
    k_mean_clf = KMeans(n_clusters = k, random_state = 42)
    k_mean_clf.fit(X)
    clusters = k_mean_clf.labels_
    centroids = k_mean_clf.cluster_centers_
    temp = sse(X,clusters, centroids)
    sse_list.append(temp)
    
sse_list    

In [ ]:
plt.plot(k_list, sse_list)